In [84]:
import sys
#!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install pytorch-ignite

    100% |████████████████████████████████| 81kB 820kB/s ta 0:00:011


In [28]:
import pandas as pd
my_path= "/home/jason/nlpws/patents/"

patent_df = pd.read_csv(my_path+'patents_nouns.csv',delimiter=',')
patent_df.columns = ["id_doc","category","concepts","words"]

train_df = pd.read_csv(my_path+'train.csv',delimiter=',', header=None, names= ["id_doc", 'category'])
dev_df = pd.read_csv(my_path+'dev.csv',delimiter=',', header=None, names= ["id_doc", 'category'])

In [60]:
train_id =train_df['id_doc'].values.tolist()
dev_id =dev_df['id_doc'].values.tolist()
#print(dev_id)

train_set = patent_df[patent_df["id_doc"].isin(train_id)]
dev_set = patent_df[patent_df["id_doc"].isin(dev_id)]
#print(train_set.shape,dev_set.shape)
print(dev_set)

train_set.to_csv(my_path+"train_set.csv", sep='\t',index=False)
dev_set.to_csv(my_path+"dev_set.csv", sep='\t',index=False)

      id_doc category                                           concepts  \
5       1001        G  bn:00000488n#bn:00007436n#bn:00021551n#bn:0007...   
6       1002        A  bn:00000489n#bn:00001698n#bn:00064167n#bn:0000...   
15      1010        A                          bn:00006835n#bn:00032726n   
24      1019        G  bn:00000488n#bn:00071525n#bn:15125301n#bn:0007...   
27      1021        G  bn:00000488n#bn:00054625n#bn:00069078n#bn:0005...   
34      1028        G  bn:00000488n#bn:01641329n#bn:15125301n#bn:0000...   
37      1030        G  bn:00000488n#bn:00061119n#bn:01325950n#bn:0007...   
41      1034        A  bn:00000490n#bn:00036926n#bn:00044489n#bn:0001...   
47       104        C  bn:00000488n#bn:00058201n#bn:00018101n#bn:0387...   
51      1043        A  bn:16998653n#bn:15125301n#bn:00019934n#bn:0001...   
58       105        C  bn:00000488n#bn:15882913n#bn:00054550n#bn:0005...   
59      1050        E  bn:00000488n#bn:00015784n#bn:00063534n#bn:0001...   
75      1065

In [62]:
#import string
def tokenizer(s):
#     table = str.maketrans({k: None for k in string.punctuation})
#     return [w.translate(table) for w in s[1:-1].split()]
    s = s.replace('#', ' ')
    return [w for w in s.split()]
def tokenizer1(s):
    return [w for w in s.split('#')] #if w.endswith('n')]


from torchtext import data as ttdata

# words, concepts and categories
txt_field = ttdata.Field(sequential=True, tokenize=tokenizer, include_lengths=True, use_vocab=True)
conc_field = ttdata.Field(sequential=True, tokenize=tokenizer1, include_lengths=True, use_vocab=True)
label_field = ttdata.Field(sequential=False, use_vocab=True, pad_token=None, unk_token=None)


tr_dev_fields = [('id_doc', None), ('category', label_field), ('concepts', conc_field), ('words', txt_field)]
train, val = ttdata.TabularDataset.splits(path=my_path, format='tsv', train='train_set.csv', validation='dev_set.csv', fields=tr_dev_fields, skip_header=True)

In [61]:
!head -n 2 /home/jason/nlpws/patents/dev_set.csv

id_doc	category	concepts	words
1001	G	bn:00000488n#bn:00007436n#bn:00021551n#bn:00076341n#bn:01831700n#bn:00021485n#bn:00041418n#bn:00079373n#bn:00047088n#bn:00047087n#bn:00005095n#bn:00007436n#bn:01831700n#bn:00021485n#bn:00047730n#bn:00051760n#bn:00017120n#bn:01831700n#bn:00021485n#bn:01831700n#bn:00021485n#bn:00076341n#bn:00007436n#bn:00021485n#bn:01732102n#bn:00047730n#bn:00060767n#bn:00047730n#bn:13713593n#bn:00047730n#bn:00040336n#bn:00062349n#bn:00021485n#bn:00027675n#bn:00062349n#bn:00021485n#bn:02690436n#bn:01831700n#bn:00021485n#bn:00005095n#bn:01105477n#bn:00054390n#bn:00031025n#bn:00061135n#bn:03303891n#bn:00034265n#bn:00021551n#bn:00021551n#bn:00064213n#bn:00021485n#bn:03632398n#bn:00059228n#bn:00041418n#bn:00079373n#bn:00047088n#bn:00047087n#bn:00021551n#bn:00059228n#bn:00026512n#bn:00007781n#bn:00021551n#bn:00079373n#bn:00047088n#bn:00047087n#bn:00021464n#bn:00021464n#bn:00005095n#bn:00020945n#bn:00079373n#bn:00047088n#bn:00047087n#bn:00077585n#bn:00079373n#bn:00016401n#

In [40]:
from torchtext import vocab
w_vec = vocab.Vectors('patent-100.vec',my_path)
c_vec = vocab.Vectors('nasari_patents.vec',my_path)

 99%|█████████▉| 55439/55894 [00:05<00:00, 9412.52it/s]

In [63]:
txt_field.build_vocab(train, val, vectors=w_vec) #max_size=100000,
conc_field.build_vocab(train, val, vectors=c_vec)
label_field.build_vocab(train)
#conc_field.vocab.itos
#txt_field.vocab.vectors.shape, txt_field.vocab.vectors[txt_field.vocab.stoi['very']]

In [86]:
import torch
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

traindl, valdl = ttdata.BucketIterator.splits(datasets=(train, val), batch_sizes=(5,3), 
                                            sort_key=lambda x: (len(x.words),len(x.concepts)), 
                                            device=device, sort_within_batch=True, repeat=False)

In [87]:
class BatchGenerator:
    def __init__(self,dl, x_wfield, x_cfield, y_field):
        self.dl, self.x_wfield, self.x_cfield, self.y_field = dl, x_wfield, x_cfield, y_field
    def __len__(self):
        return len(self.dl)
    def __iter__(self):
        for b in self.dl:
            xw = getattr(b, self.x_wfield)
            xc = getattr(b, self.x_cfield)
            y = getattr(b, self.y_field)
            yield((xw, xc), y)
train_batch_it = BatchGenerator(traindl, 'words', 'concepts','category')

In [88]:
import torch
import torch.nn as nn
SEED = 789 #123456
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
class ConcatPoolingGRU(nn.Module):
    def __init__(self, wvocab_sz, wemb_dim,cvocab_sz, cemb_dim, n_hidden, n_out, wpre_vec, cpre_vec,bidirectional=True):
        super().__init__()
        self.wvocab_sz = wvocab_sz
        self.cvocab_sz = cvocab_sz
        self.wemb_dim = wemb_dim
        self.cemb_dim = cemb_dim
        self.n_hidden = n_hidden
        self.n_out = n_out
        
        self.wemb = nn.Embedding(self.wvocab_sz, self.wemb_dim)
        self.wemb.weight.data.copy_(wpre_vec)
        self.wemb.weight.requires_grad = False
        self.cemb = nn.Embedding(self.cvocab_sz, self.cemb_dim)
        self.cemb.weight.data.copy_(cpre_vec)
        self.cemb.weight.requires_grad = False
        
        self.bidirectional = bidirectional
        
        self.wgru = nn.GRU(self.wemb_dim,self.n_hidden, bidirectional=bidirectional)
        self.cgru = nn.GRU(self.cemb_dim,self.n_hidden, bidirectional=bidirectional)
        self.lin1 = nn.Linear(300, self.n_hidden)
        self.nlin1 = nn.ReLU()
        self.drop = nn.Dropout(0.1)
        if bidirectional:
            self.out = nn.Linear(self.n_hidden*2*2, self.n_out)
        else:
            self.out = nn.Linear(self.n_hidden*4, self.n_out)
            
    def forward(self, seq):
#         bs = seq.size(1)
        w_seq, c_seq = seq
        w_seq,wl_seq = w_seq
        wl_seq, _ = torch.sort(wl_seq, descending=True)
#         print(wl_seq)
        c_seq,cl_seq = c_seq
        bs = w_seq.size(1)
        self.hw = self.init_hidden(bs)
        self.hc = self.init_hidden(bs)
        
        w_seq = w_seq.transpose(0,1)
        c_seq = c_seq.transpose(0,1)
        
        wembs = self.wemb(w_seq)
        wembs = wembs.transpose(0,1)
        wembs = nn.utils.rnn.pack_padded_sequence(wembs, wl_seq)
        
        cembs = self.cemb(c_seq)
#         cembs = cembs.transpose(0,1)
#         cembs = nn.utils.rnn.pack_padded_sequence(cembs, torch.sort(cl_seq, descending=True)[0])
        cembs = cembs.mean(dim=1)
#         print(cembs.shape)
        cembs = self.lin1(cembs)
        cembs = self.drop(cembs)
        cembs = self.nlin1(cembs)
        
        cembs = cembs.view(bs,-1)
#         cembs = cembs.transpose(0,1)
        wgru_out, self.hw = self.wgru(wembs, self.hw)
        wgru_out, _ = nn.utils.rnn.pad_packed_sequence(wgru_out)
#         print(self.hw.shape)
#         cgru_out, self.hc = self.cgru(cembs, self.hc)
#         cgru_out, _ = nn.utils.rnn.pad_packed_sequence(cgru_out)
        
        wavg_pool = nn.functional.adaptive_avg_pool1d(wgru_out.permute(1,2,0),1).view(bs, -1)
        wmax_pool = nn.functional.adaptive_avg_pool1d(wgru_out.permute(1,2,0),1).view(bs,-1)
        wmax_pool = self.drop(wmax_pool)
        wmax_pool = self.nlin1(wmax_pool)
        wavg_pool = self.drop(wavg_pool)
        wavg_pool = self.nlin1(wavg_pool)
#         cavg_pool = nn.functional.adaptive_avg_pool1d(cgru_out.permute(1,2,0),1).view(bs,-1)
#         cmax_pool = nn.functional.adaptive_avg_pool1d(cgru_out.permute(1,2,0),1).view(bs,-1)
        
        outp = self.out(torch.cat([torch.squeeze(self.hw),wavg_pool,wmax_pool,cembs],dim=1))
        return nn.functional.log_softmax(outp, dim=-1)
        
    def init_hidden(self, batch_sz):
        if self.bidirectional:
            return torch.zeros(2, batch_sz, self.n_hidden).to(device)
        else:
            return torch.zeros(1,batch_sz, self.n_hidden).to(device)
    

In [89]:
wvocab_sz = len(txt_field.vocab)
wemb_dim = 100
cvocab_sz = len(conc_field.vocab)
cemb_dim = 300

n_hidden = 64
n_out = len(label_field.vocab)

traindl, valdl = data.BucketIterator.splits(datasets=(train, val), batch_sizes=(8,16), sort_key=lambda x:(len(x.words),len(x.concepts)), device=device, sort_within_batch=True,repeat=False)
train_batch_it = BatchGenerator(traindl,'words','concepts','category')
val_batch_it = BatchGenerator(valdl, 'words','concepts','category')
m = ConcatPoolingGRU(wvocab_sz,wemb_dim,cvocab_sz,cemb_dim, n_hidden, n_out, train.fields['words'].vocab.vectors,train.fields['concepts'].vocab.vectors, bidirectional=False).to(device)
opt = torch.optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), 1e-2)
criterion = nn.NLLLoss()
# fit model with ignite

In [91]:
from ignite.engine import Engine, Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss, RunningAverage, Precision, Recall, MetricsLambda
from ignite.handlers import ModelCheckpoint,  EarlyStopping
# def process_function(engine, batch):
#     m.train()
#     optimizer.zero_grad()
#     x, y = batch.content, batch.label
#     y_pred = m(x)
#     loss = criterion(y_pred, y)
#     loss.backward()
#     opt.step()
#     return loss.item()

# def eval_function(engine, batch):
#     m.eval()
#     with torch.no_grad():
#         x, y = batch.content, batch.label
#         y_pred = m(x)
#         return y_pred, y
    
# trainer = Engine(process_function)
precision = Precision(average=False)
recall = Recall(average=False)
F1 = precision*recall*2/(precision+recall+1e-20)
F1 = MetricsLambda(lambda t: torch.mean(t).item(), F1)
trainer = create_supervised_trainer(m, opt, criterion)
# train_evaluator = Engine(eval_function)
evaluator = create_supervised_evaluator(m, metrics={'accuracy':Accuracy(),'nll': Loss(criterion), 'f1':F1})
# validation_evaluator = Engine(eval_function)

# RunningAverage(output_transform=lambda x: x).attach(trainer, 'loss')

@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    evaluator.run(train_batch_it)
    metrics = evaluator.state.metrics
    print("Training Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f} f1: {:.2f}"
          .format(trainer.state.epoch, metrics['accuracy'], metrics['nll'], metrics['f1']))

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(val_batch_it)
    metrics = evaluator.state.metrics
    print("Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f} f1: {:.2f}"
          .format(trainer.state.epoch, metrics['accuracy'], metrics['nll'], metrics['f1']))

trainer.run(train_batch_it, max_epochs=5)


Training Results - Epoch: 1  Avg accuracy: 0.83 Avg loss: 0.45 f1: 0.81
Validation Results - Epoch: 1  Avg accuracy: 0.70 Avg loss: 0.92 f1: 0.63
Training Results - Epoch: 2  Avg accuracy: 0.86 Avg loss: 0.36 f1: 0.86
Validation Results - Epoch: 2  Avg accuracy: 0.70 Avg loss: 0.94 f1: 0.59
Training Results - Epoch: 3  Avg accuracy: 0.85 Avg loss: 0.40 f1: 0.83
Validation Results - Epoch: 3  Avg accuracy: 0.69 Avg loss: 1.04 f1: 0.62
Training Results - Epoch: 4  Avg accuracy: 0.82 Avg loss: 0.46 f1: 0.82
Validation Results - Epoch: 4  Avg accuracy: 0.66 Avg loss: 1.05 f1: 0.54
Training Results - Epoch: 5  Avg accuracy: 0.83 Avg loss: 0.44 f1: 0.78
Validation Results - Epoch: 5  Avg accuracy: 0.69 Avg loss: 0.97 f1: 0.58
